# SMSSpamCollection
# Location : https://mitu.co.in/dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
#pip install keras_preprocessing

In [61]:
import string
import re
from os import listdir
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, Flatten
#from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [62]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [63]:
load_doc('/content/drive/MyDrive/CDAC_DAI/NLP/review_polarity/txt_sentoken/neg/cv000_29416.txt') #Demo

'plot : two teen couples go to a church party , drink and then drive . \nthey get into an accident . \none of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . \nwhat\'s the deal ? \nwatch the movie and " sorta " find out . . . \ncritique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . \nwhich is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn\'t snag this one correctly . \nthey seem to have taken this pretty neat concept , but executed it terribly . \nso what are the problems with the movie ? \nwell , its main problem is that it\'s simply too jumbled . \nit starts off " normal " but then downshifts into this " fantasy " world in which you , as an audience membe

In [64]:
# turn a doc into clean token
def clean_doc(doc):
  # split into tokens by white space
  tokens = doc.split()
  # prepare regex for char filtering
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))
  # remove punctuation from each word
  tokens = [re_punc.sub('', w) for w in tokens]
  # remove remaning tokens that are not aplhatic
  tokens = [word for word in tokens if word.isalpha()]
  # Filter out stop words
  stop_words = set(stopwords.words('english'))
  tokens = [w for w in tokens if not w in stop_words]
  # filter out short tokens
  tokens = [word for word in tokens if len(word) > 1]
  return tokens

In [65]:
text = load_doc('/content/drive/MyDrive/CDAC_DAI/NLP/review_polarity/txt_sentoken/neg/cv000_29416.txt')
#clean_doc(text)

In [66]:
# # load doc, clean and return line of token
# def doc_to_line(filename, vocab):
#   # load the doc
#   doc = load_doc(filename)
#   # clean doc
#   tokens = clean_doc(doc)
#   # filter by vocab
#   tokens = [w for w in tokens if w in vocab]
#   return ' '.join(tokens)

In [67]:
# vocab = open('/content/drive/MyDrive/CDAC_DAI/NLP/vocab.txt', 'r')
# vocab = set(vocab.read().split())

In [68]:
#doc_to_line('/content/drive/MyDrive/CDAC_DAI/NLP/review_polarity/txt_sentoken/neg/cv000_29416.txt', vocab)

In [69]:
# # load all documents in a directory
# def process_train(directory):
#   document =list()
#   for filename in listdir(directory):
#     # create the full path of the file to open
#     if not filename.startswith('cv9'):
#       path = directory + '/' + filename
#       # load and clean the doc
#       doc = load_doc(path)
#       tokens = clean_doc(doc)
#       # add to list
#       document.append(tokens)
#   return document


#   # load all documents in a directory
# def process_test(directory):
#   document =list()
#   for filename in listdir(directory):
#     # create the full path of the file to open
#     if not filename.startswith('cv9'):
#       path = directory + '/' + filename
#       # load and clean the doc
#       doc = load_doc(path)
#       tokens = clean_doc(doc)
#       # add to list
#       document.append(tokens)
#   return document







In [70]:
#process_train('/content/drive/MyDrive/CDAC_DAI/NLP/review_polarity/txt_sentoken/neg')

In [71]:
# load all doc in directory
def process_doc(directory, is_train):
  documents = list()
  for filename in listdir(directory):
    # skip any review in the test set
    if is_train and filename.startswith('cv9'):
      continue
    if not is_train and not filename.startswith('cv9'):
      continue
    # create the full path of the file to open
    path = directory + '/' + filename
    # load and clean the doc
    doc = load_doc(path)
    tokens = clean_doc(doc)
    # add to list
    documents.append(tokens)
  return documents

In [145]:
#process_doc('/content/drive/MyDrive/CDAC_DAI/NLP/review_polarity/txt_sentoken/neg', True)

In [73]:
# load and clean a dataset
def load_clean_dataset(is_train):
  # load documents
  neg = process_doc('/content/drive/MyDrive/CDAC_DAI/NLP/review_polarity/txt_sentoken/neg', is_train)
  pos = process_doc('/content/drive/MyDrive/CDAC_DAI/NLP/review_polarity/txt_sentoken/pos', is_train)
  documents = neg + pos
  # prepare labels
  labels = [0 for _ in range(len(neg))] + [1 for _ in range(len(pos))]
  return documents, labels

In [74]:
train, train_labels = load_clean_dataset(True)
test, test_labels = load_clean_dataset(False)

In [75]:
len(train), len(train_labels)

(1810, 1810)

In [76]:
len(test), len(test_labels)

(200, 200)

In [77]:
train_labels[100]

0

In [78]:
from collections import Counter
Counter(train_labels)

Counter({0: 910, 1: 900})

In [79]:
Counter(test_labels)

Counter({0: 100, 1: 100})

In [80]:
# fit a tokenizer
def create_tokenizer(lines):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer


# integer encode and pad documwnts
def encode_docs(tokenizer, max_length, docs):
  # integer encode
  encoded = tokenizer.texts_to_sequences(docs)
  # pad encoded sequences
  padded = pad_sequences(encoded, maxlen=max_length, padding='post')
  return padded

Build the model

In [81]:
# define the model
def define_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 100, input_length=max_length))
  model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # compile network
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  # summarize defined model
  return model

In [82]:
# Load all reviews
train_docs, train_labels = load_clean_dataset(True)
test_docs, test_labels = load_clean_dataset(False)

In [83]:
len(train_docs), len(test_docs)

(1810, 200)

In [118]:
# Create the tokenizer
tokenizer = create_tokenizer(train_docs)

In [119]:
# Define vocablary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)

Vocabulary size: 44277


In [137]:
# Calculated the mximum sequenc length
max_length = max([len(s) for s in train_docs])
print('Max document length: %d' % max_length)

Max document length: 1380


In [138]:
# Encode the documents
Xtrain = encode_docs(tokenizer, max_length, train_docs)
Xtest = encode_docs(tokenizer, max_length, test_docs)

In [139]:
Xtrain.shape, Xtest.shape

((1810, 1380), (200, 1380))

Define the model


In [140]:
model = define_model(vocab_size, max_length)


In [141]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1380, 100)         4427700   
                                                                 
 conv1d_2 (Conv1D)           (None, 1373, 32)          25632     
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 686, 32)           0         
 g1D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 21952)             0         
                                                                 
 dense_4 (Dense)             (None, 10)                219530    
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                      

Train Model

In [142]:
# fit network
model.fit(Xtrain, np.array(train_labels), epochs=10, batch_size=10)

Epoch 1/10
181/181 [==============================] - 23s 119ms/step - loss: 0.6934 - accuracy: 0.5039
Epoch 2/10
181/181 [==============================] - 16s 89ms/step - loss: 0.6753 - accuracy: 0.5199
Epoch 3/10
181/181 [==============================] - 12s 67ms/step - loss: 0.4037 - accuracy: 0.8133
Epoch 4/10
181/181 [==============================] - 10s 53ms/step - loss: 0.0551 - accuracy: 0.9818
Epoch 5/10
181/181 [==============================] - 10s 55ms/step - loss: 0.0058 - accuracy: 0.9983
Epoch 6/10
181/181 [==============================] - 9s 50ms/step - loss: 0.0031 - accuracy: 0.9994
Epoch 7/10
181/181 [==============================] - 6s 31ms/step - loss: 0.0023 - accuracy: 0.9994
Epoch 8/10
181/181 [==============================] - 6s 35ms/step - loss: 0.0017 - accuracy: 0.9994
Epoch 9/10
181/181 [==============================] - 5s 26ms/step - loss: 9.2270e-04 - accuracy: 0.9994
Epoch 10/10
181/181 [==============================] - 5s 27ms/step - loss: 3.759

Evaluate on test data

In [98]:
model.evaluate(Xtest, np.array(test_labels), batch_size=1)

200/200 [==============================] - 1s 3ms/step - loss: 0.6932 - accuracy: 0.5000


[0.6931553483009338, 0.5]

Predict on unseen data

In [127]:
text1 = 'Best movie ever! It was great, I will definitely recommend it.'
text2 = 'This is a bad movie. Please dont watch it.'

In [95]:
# # create the tokenizer
# tokenizer1 = create_tokenizer(text1)
# tokenizer2 = create_tokenizer(text2)


In [106]:
# # encode data
# x_text1 = tokenizer1.texts_to_matrix(text1)
# #encode data
# x_text2 = tokenizer2.texts_to_matrix(text2)

In [134]:
# Classify a revew as negative or positive
def predict_sentiment(review):
  # clean the data
  tokens = clean_doc(review)
  # create the string
  line = ' '.join(tokens)
  encoded = encode_docs(tokenizer, max_length, [line])
  # predict sentiment
  yhat = model.predict(encoded, verbose=0)
  # retrive pridicted percentage and leabel
  percentage_pos = yhat[0,0]
  if round(percentage_pos) == 0:
    return (1-percentage_pos), 'NEGATIVE'
  return percentage_pos, 'POSITIVE'

In [143]:
predict_sentiment(text1)

(0.57932246, 'POSITIVE')

In [144]:
predict_sentiment(text2)

(0.5761577785015106, 'NEGATIVE')

In [1]:
training_sentence  =  []
training_labels = []
test_sentence = []
test_labels = []
for s,l in train_data:
  training_sentence.append(str(s.numpy()))
  training_labels.append(l.numpy)
for s,l in test_data:
  test_sentence.append(str(s.numpy()))
  test_labels.append(l.numpy)

NameError: name 'training' is not defined